In [ ]:
%pip install openai
%pip install --upgrade openai
%pip install langchain
%pip install PyPDF2
%pip install --upgrade --quiet  langchain-openai

In [ ]:
import openai
from langchain.chat_models import ChatOpenAI
import os
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import re
from PyPDF2 import PdfReader
import pandas as pd
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from dotenv import load_dotenv
import pandas as pd


load_dotenv()

In [ ]:

deployment_name=os.getenv("deployment_name")#'gpt-4-32k-rfmanrique'
version="2023-05-15"

api_key= os.getenv("api_key")
openai_api_key = api_key

api_version="2023-12-01-preview"
azure_endpoint="https://invuniandesai-2.openai.azure.com/"


llm = AzureChatOpenAI(deployment_name=deployment_name, openai_api_version=version, openai_api_key=api_key, openai_api_base=azure_endpoint)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://invuniandesai-2.openai.azure.com/ to https://invuniandesai-2.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `de

In [ ]:


def extract_text_from_pdf(file_path, page_numbers=None):
    """Extracts text from specified pages of a PDF."""
    reader = PdfReader(file_path)
    text = ""
    if page_numbers is None:
        page_numbers = range(len(reader.pages))
    for i in page_numbers:
        text += reader.pages[i].extract_text() + '\n'
    return text

def find_numbers_in_text(text, pattern=r'\((-?\d+(\.\d+)?)\)'):
    """Finds numbers in the given text based on the specified regular expression pattern."""
    matches = list(re.finditer(pattern, text))
    return [(int(m.group(1)), m.start()) for m in matches]

def extract_text_first_parentheses(text):
    # Regular expression pattern to find text within the first pair of parentheses
    pattern = r'\((.*?)\)'
    match = re.search(pattern, text)
    if match:
        # Return the text found within the first pair of parentheses
        return match.group(1)
    else:
        # If no parentheses are found, return None or an appropriate message
        return None

def extract_articles(texts, articles):
    """Extracts specified articles from the text."""
    result = {}
    for index, (text_snippet, article_number) in enumerate(zip(texts, articles)):
        try:
            # Assuming 'llm_chain.run' and 'PromptTemplate' are defined elsewhere
            template + text_snippet
            gpt_response = llm_chain.run(article=article_number, text=text_snippet)

            articles = [language_response.split(' | ')[1] for language_response in gpt_response.split('\n')]
            result[index] = {
                'es': articles[0],
                'wy': articles[1]
            }
        except Exception as e:
          # Code that will run if there is an exception
          print("An exception occurred:", e)
          print(gpt_response)
    return result


def get_text_segments(text, div_word):
  segments = text.split(div_word)[1:]
  segments = [div_word + segment[:-len(div_word)] for segment in segments]
  segments_article_number = [extract_text_first_parentheses(segment) for segment in segments]
  return segments,segments_article_number

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://invuniandesai-2.openai.azure.com/ to https://invuniandesai-2.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `de

In [ ]:
pdfs_info = {
    '72.  Vamos a hablar tomo II. WEB.pdf':
      {
          'start_page': 16,
          'end_page': 100
      },
    '8548776.pdf':
      {
          'start_page': 14,
          'end_page': 46
      },
    'pdfcoffee.com_la-conjugacion-del-verbo-en-la-lengua-wayuu-pdf-free.pdf':
      {
          'start_page': 1,
          'end_page': 33
      },
    'Compendio de la Gramática de la Lengua Wayuu ( PDFDrive ).pdf':
      {
          'start_page': 15,
          'end_page': 112
      },


}


In [ ]:
pdf_to_check =  'Compendio de la Gramática de la Lengua Wayuu ( PDFDrive ).pdf'

In [ ]:
template = """
Tengo este texto busca regiones en el texto donde aparezcan oraciones en wayuu y español y haz una tabla evita casas como "a’wataa" - "a’waataa" que son mas bien palabras considera que es un libro de enseñansa del idioma

(el output en formato csv con las columas: wayuu, español) sin usar codigo
ejemplo:
"wayuu","español"
"Kakaa’ulainshi Kamiirü","Camilo tiene chivos."
"Makaa’ulain salii tepichikana","Los niños no tienen chivos."

Texto:
"""

In [ ]:
results = []


In [ ]:
n_pages_p_iterations = 3
n_iteration = (pdfs_info[pdf_to_check]['end_page']-pdfs_info[pdf_to_check]['start_page'])//n_pages_p_iterations
text_queries = []
for i in range(n_iteration):
  try:

    pdf_file_path = pdf_to_check

    full_text = extract_text_from_pdf(pdf_file_path, page_numbers=range(pdfs_info[pdf_to_check]['start_page']+i*n_pages_p_iterations, pdfs_info[pdf_to_check]['start_page']+(i+1)*n_pages_p_iterations))

    msg = HumanMessage(content=template + full_text)
    llm = AzureChatOpenAI(deployment_name=deployment_name, openai_api_version=version, openai_api_key=api_key, openai_api_base=azure_endpoint)

    gpt_response = llm(messages=[msg]).content
    csv_clean = [i[1:-1].split('","') for i in gpt_response.split('\n')][1:]
    csv_clean = [[i[0],i[1]]for i in csv_clean]
    df = pd.DataFrame(csv_clean, columns=['wy','es'])
    display(df)
    results.append(df)
  except Exception as e:
    pass

/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://invuniandesai-2.openai.azure.com/ to https://invuniandesai-2.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.com/openai/dep

,wy,es
0,Washitshaanas hi ma’i wayuukai chii,Este hombre es muy rico
1,Paapaa susukoo tüü sümaa ja'rai shikii nneerü,Toma la mochila esta con cincuenta monedas
2,"Japüishi chira jintüi kai, nnojoishi ekeein o...","Ese niño tiene verg üenza, no quiere comer del..."
3,Akuyamajüshi taya türa luma kat,Yo construí es a enramada
4,¿Ashajayaashikalej e'e chira Ø?,¿Ése está escribiendo o no?
5,Ana putuma türa Ø checheiwa kojonkolu,Acomoda bien eso para que el cajón quede apretado
6,"Ayonnajüshi Kamiirü (taya, nia, pia)","Camilo baila (yo bailo, él baila, tú bailas )"
7,"Ayonnajüsü Mariia (taya, shia, pia)","María baila (yo bailo, ella baila, tú bailas )"
8,"Ayonnajüshii wayuukana (waya, naya, jaya)","Los wayuu bailan (nosotros, ellos, Uds.)"
9,"A’lakajaashi Kamiirü (taya, nia, pia)","Camilo cocina (yo cocino , él cocina , tú coc..."


/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://invuniandesai-2.openai.azure.com/ to https://invuniandesai-2.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.com/openai/dep

,wy,es
0,apüta a apüta - ...,dejar ...
1,achaja waa achaja - achajaa -...,buscar ...
2,jimata a jimata - ...,estar quieto ...
3,kamane waa kamane - kamanee - ...,ser amable ...
4,Tachajaainjatü tü saruweesa kalü,Yo voy a buscar la cerveza. (Específico)
5,Achajaainjachi taya saruweesa,Yo voy a buscar cerveza. (Específico)
6,Püya’lajeeerü tü meruuna kalü,Tú comprarás el melón . (Específico)
7,Aya’lajeerü pia meruuna,Tú comprarás melón . (Específico)
8,Wa’lakajaa injatü tü asalaa kalü,Vamos a cocinar la carne. (Específico)
9,A’lakajaa injana waya asalaa,Vamos a cocinar carne. (Específico)


/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://invuniandesai-2.openai.azure.com/ to https://invuniandesai-2.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.com/openai/dep

...

An exception occurred: list index out of range
"wayuu","español"
"O'otojoolii waya washe'ein [jüpüla atunkaa ].", "Nosotros solemos sacudir nuestra ropa antes de dormir."
"O’unushi pia [süpülapünaa nantüin tepichikana ].", "Tú te marchaste antes de que llegaran los niños."
"[Süpülapünaa temi'ijaain jamaa ], acheküsü ta'liimajüin japato'ushekalüirua.", "Antes de yo jugar con ustedes,  tengo que limarles las uñas."
"O'unushi wayuukai [ayuuliiwa'aya tü niikalü ].", "El hombre se  fue cuando todavía estaba enferma su madre."
"Yootomuusu tü wayuukoluirua [jutkataiwa'aya jia wüinroku ].", "Las personas conversaban mientras se encontraban en el abrevadero."
"Onjula'alaa müsü wayuukolu [jemeyuluiwa'aya ji'rüin tü juwalakalü ].", "La mujer se escondió cuando vio que estaba pariendo su hermana."
"Aya'lajüshi taya jüshe'ein tachon, [nnojoluiwa'aya kojutuin tü kasakalü ].", "Yo compré la ropa de mi hija cuando todavía no estaban caras las cosas."
"[Müle’u pa pia], müliap u’ujeechi pia jutuma tü wü

/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://invuniandesai-2.openai.azure.com/ to https://invuniandesai-2.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.com/openai/dep

,wy,es
0,Naashajaapu'u amaalü nu’wayuusekalü [jii'iree ...,Le hablaba frecuentemente a ella para que lo a...
1,Warai müshi wayuukai jainküinpünaa tü a'ttieek...,El hombre caminaba por entre el sembradío con ...
2,"Jiasa'a jo'u wanee ka'i, antataalashi nainchi ...","Entonces un día, llegó el cuñado de Kushematai..."
3,Tawalaajeerü tojuyaala namüin [suulia neiwa’aj...,Les pagaré mi deuda a ellos para que no me fas...
4,Apantajaashi chi ka’ruwaraikai [juulia ata’ünn...,El ladrón salió corriendo para no ser atrapado...
5,Nüsirüin laülaakai wanee wunu'u [suulia mojuja...,El viejo le hizo beber un remedio para que no ...
6,Anujulaashi chi kaainjaraikai shiyaama'a w ane...,El malhechor se escondió detrás de un árbol pa...
7,Puchuwajaa siki [ta'lakajaa iwa piküin ].,Enciende el fuego para cocinarte tu comida.
8,Anakaja’a to’unule eemüin Jusepiina [tachunta ...,Es bueno que yo vaya a donde Josefina para ped...
9,Joo'uya amouyuwoumüin [pü'yalaja iwa jü'ütpa'a...,Vamos al cementerio para que llores junto a tu...


/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://invuniandesai-2.openai.azure.com/ to https://invuniandesai-2.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.com/openai/dep

,wy,es
0,Talatüshi taya süka te'raajü in toumain; talat...,Estoy contento porque conozco mi tierra; estoy...
1,"Niya’lajüinjatü eküülü , süka saja’lajaa in","Él va a comprar comida, porque se acabó."
2,"Japüishii jaya, süka mulo'uyuushaana in türa j...","Ustedes tienen vergüenza, porque tienen las uñ..."
3,"Teküin süchon tü wunu’ukot , süka shimeejü in ...","Yo comí el fruto del árbol, porque la serpient..."
4,A'yalajüshi taya aka ju'unuinjatü in tawala,Yo estoy llorando porque mi hermana se va a ma...
5,A'ülüjüna amüinrü jietkalü aka jantirü in chi ...,La mujer fue reprendida porque había llevado a...
6,"Anakaja tashapajaale , aka sa’atapajü in taya ...","Es conveniente que me apresure, porque me está..."
7,"Nnojotsü anain jüpüla pu'wayuusein taya , aka ...","No conviene que yo sea tu mujer, porque no tú ..."
8,"Nnojoleerü tejeerajüin jüyaakuwa takatünasüin,...","No dibujaré mi avión, por que es difícil para mí."
9,E’rüi wayumüin pia sükajee paa’inrü in tia wamüin,"Tú eres una mala persona, porque nos hiciste eso."


In [ ]:
df = pd.concat(results).reset_index(drop = True)

In [ ]:
df['source'] = pdf_to_check

In [ ]:
df.to_csv(f'{pdf_to_check}.csv', index=False)
